In [1]:
import pandas as pd
import re
import numpy as np
import requests
import os
from tqdm import tqdm
import urllib
import rembg
from PIL import Image
from matplotlib import pyplot as plt
import cv2
import urllib.request
from concurrent.futures import ThreadPoolExecutor, as_completed
import tempfile
import configparser
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import time
from tqdm import tqdm
import pandas as pd
import urllib.request
from concurrent.futures import ThreadPoolExecutor, as_completed
import tempfile
import configparser
import rembg
from PIL import Image

In [2]:
config = configparser.RawConfigParser()
config.read('config.ini')
img_folder = config['folder_name']['img_download']
product_table_path = config['intermediate_path']['product_table']
image_table_path = config['intermediate_path']['image_table']
os.makedirs(img_folder, exist_ok=True)

In [3]:
# def resize_image(img, output_size=(256, 256)):
#     return img.resize(output_size, Image.LANCZOS)

# def process_image(product_id, img_url):
#     try:
#         # Create a temporary file for each image
#         with tempfile.NamedTemporaryFile(delete=True, suffix='.png') as temp_file:
#             urllib.request.urlretrieve(img_url, temp_file.name) # download temp file

#             img = Image.open(temp_file.name)
#             rm_back = rembg.remove(img) # Remove background

#             if rm_back.mode == 'RGBA':
#                 rm_back = rm_back.convert('RGB') # Convert RGBA -> RGB

#             output = resize_image(rm_back)
#             output.save(f'{img_folder}/{product_id}.png', 'PNG')

#             return {'product_id': {product_id}, 'image_path': f'{img_folder}/{product_id}.png'}
    
#     except Exception as e:
#         print(f'Error processing {product_id}: {e}')
#         return None
    
# def create_img_df(img_url_list, max_workar=8):
#     try:
#         rows = []

#         with ThreadPoolExecutor(max_workers=max_workar) as executor:
#             features = {executor.submit(process_image, pid, url): pid for pid, url in img_url_list}

#             for feature in tqdm(as_completed(features), total=len(img_url_list)):
#                 result = feature.result()
#                 if result:
#                     rows.append(result)
        
#         return pd.DataFrame(rows)
#     except Exception as e:
#         raise e

In [4]:
# img_df = pd.read_csv('product_table.csv')
# img_df.head()

In [5]:
# df1 = img_df.groupby('product_name').agg('count')
# df1.head()

In [6]:
def open_browser(path):
    try:
        # set up Selenium WebDriver
        driver = webdriver.Chrome()
        driver.get(path)

        # Wait for page to load
        time.sleep(20)
        
        driver.find_element(By.XPATH, '//button[text() ="Ask Me Later"]').click()
        return driver
    
    except Exception as e:
        raise e
    
def resize_image(img, output_size=(256, 256)):
    return img.resize(output_size, Image.LANCZOS)

def process_image(driver, product_id, img_url, img_extension=['.jpeg'], folder_name=img_folder):
    try:
        images = driver.find_elements(By.TAG_NAME, 'img')
        img_url = list(
            filter(
                lambda url: any(extn in url for extn in img_extension),
                map(lambda img: img.get_attribute('src'), images)
            )            
        )[0]

        # Create a temporary file for each image
        with tempfile.NamedTemporaryFile(delete=True, suffix='.png') as temp_file:
            urllib.request.urlretrieve(img_url, temp_file.name) # download temp file

            img = Image.open(temp_file.name)
            rm_back = rembg.remove(img) # Remove background

            if rm_back.mode == 'RGBA':
                rm_back = rm_back.convert('RGB') # Convert RGBA -> RGB

            output = resize_image(rm_back)
            output.save(f'{folder_name}/{product_id}.png', 'PNG')

            return {'product_id': product_id, 'image_path': f'{folder_name}/{product_id}.png'}
    
    except Exception as e:
        print(f'Error processing {product_id}: {e}')
    
    finally:
        driver.quit()

def process_wrapper(id, path):
    try:
        driver = open_browser(path=path)
        row = process_image(driver, id, path, img_extension=['.jpeg'], folder_name=img_folder)
        return row
    except Exception as e:
        raise e

def save_img(prod_dict):
    try:
        rows = []

        for item in tqdm(prod_dict.items()):
            results = process_wrapper(*item)
            
        rows.extend(results)
        return pd.DataFrame([r for r in rows if r])
    
    except Exception as e:
        raise e

In [7]:
prod_href = pd.read_csv(product_table_path)
prod_id_index = prod_href[['product_id', 'product_url']].set_index('product_id').to_dict('index')
prod_id_href_dict = dict(map(lambda x: (x, list(prod_id_index[x].values())[0]), prod_id_index.keys()))

In [10]:
k =[]
for key, value in zip(list(prod_id_href_dict.keys())[907:1000], list(prod_id_href_dict.values())[907:1000]): 
    k.append((key, value))                                                                                                                                                                                                                                                                                                                                      
dic = dict(k)                                                                                                                                                                                                                                                                                                                                                                                                                                             
# dic                                                                 

In [11]:
df = save_img(dic)

  0%|          | 0/93 [00:35<?, ?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[text() ="Ask Me Later"]"}
  (Session info: chrome=133.0.6943.141); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x64438a29714a <unknown>
#1 0x644389d34b80 <unknown>
#2 0x644389d860e9 <unknown>
#3 0x644389d86271 <unknown>
#4 0x644389dd4de4 <unknown>
#5 0x644389dabefd <unknown>
#6 0x644389dd214a <unknown>
#7 0x644389dabca3 <unknown>
#8 0x644389d77f08 <unknown>
#9 0x644389d79071 <unknown>
#10 0x64438a260b5b <unknown>
#11 0x64438a264ae2 <unknown>
#12 0x64438a24c967 <unknown>
#13 0x64438a2656d4 <unknown>
#14 0x64438a230c7f <unknown>
#15 0x64438a285cd8 <unknown>
#16 0x64438a285ea9 <unknown>
#17 0x64438a295fc6 <unknown>
#18 0x78a9a729caa4 <unknown>
#19 0x78a9a7329c3c <unknown>


In [14]:
len(list(prod_id_href_dict.keys())[47:100])

53